I have run the code using 'Bigram'. When I change the variable 'ngramRange' to 1, Code runs using 'Unigram'. You can see accuracy of Unigram at the end of the report.

As you can see below, Firstly, I read the data line by line and stored in a Dataframe 'all data'.
'all_data''s columns are topic, label, doc_identifier and text. You can see first 5 elements of all_data below.

In [1]:
import pandas as pd
import time

f = open("all_sentiment_shuffled.txt",'r', encoding="utf8")
lines = f.readlines() # read the data line by line and store list lines

topics = []
labels = []
doc_identifiers = []
text = []

for line in lines:
    line.lower() # make all characters lowercase
    splited_line = line.split() # split the line to spaces
    topics.append(splited_line[0]) # first splited word is topic
    labels.append(splited_line[1]) # second splited word is label of sentence
    doc_identifiers.append(splited_line[2]) # third splited word is name of the file
    for i in splited_line[3:]: # rest of them is sentence
        if not i.isalpha(): # remove chars that is not in alphabet
            splited_line.remove(i)
    sentence = ' '.join(splited_line[3:]) # combine words again
    text.append(sentence)

# akeeps the all data into dataframe all_data
dict = {"topic":topics, "label":labels, "doc_identifier":doc_identifiers, "text":text}
all_data = pd.DataFrame(dict)
all_data.head()

,doc_identifier,label,text,topic
0,241.txt,neg,i bought this album because i loved the title ...,music
1,544.txt,neg,i was misled and thought i was buying the enti...,music
2,729.txt,neg,i have introduced many of my ell high school s...,books
3,278.txt,pos,anything you purchase in the left behind serie...,books
4,840.txt,pos,i loved these movies and i cant wiat for the t...,dvd


Then, I splitted the data into 3 parts. Firstly, it is splitted as test and train data. Test/data = 1/5
After that, I separeted datas based on whether it has a positive or negative label for easier implementation.
I don't shuffle the data because the data is already shuffled.

In [2]:
from sklearn.model_selection import train_test_split

#split data into train and test.
train, test = train_test_split(all_data, test_size=0.2, shuffle=False)
train_pos = train[train['label'] == "pos"]
train_neg = train[train['label'] == "neg"]


I used Bag of Words model for counting the number of times each word appears. My implementation of BOW is in the below.
ngramRange is 2 now, So, my code is running as Bigram Model. I we change to 1, it would run as Unigram Model. I run with Unigram model before. Details are at the end of the file.

English Stop Words are words that using commonly. For example, pronouns, prepositions, conjunctions...
When I remove them, My code works more speedy as well as accuracy increases.

Vectorizer lists all words in data that have a positive label. And it creates a matrix. Matrix keeps that each word's number of occurrence for each data sample.
I create a new Dataframe through that matrix. New Dataframe's columns are vocabularies and its number of occurance from the matrix.

!For Bigram Model, "Vocabulary" is a structure that occurred as getting together 2 words.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS, TfidfVectorizer
import numpy as np

ngramRange = 2 # it can be change according to be unigram or bigram

# remove stopwords and create unigram or bigram
vectorizer_pos = CountVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(ngramRange,ngramRange))
x = vectorizer_pos.fit_transform(train_pos.loc[:,"text"])
bow_matrix_pos = x.toarray() # create a bow matrix that keeps occurances of all words in positive data

pos_vocabularies = vectorizer_pos.get_feature_names() # all words in positive data
numberOfVoc_pos = np.sum(bow_matrix_pos,axis=0) # number of occurrency of vocabularies that have positive label
dict_pos = {"positive vocabularies":pos_vocabularies, "number of occurrency":numberOfVoc_pos}
pos_data = pd.DataFrame(dict_pos) # data that have words and their occurrency number
pos_data.head()

,number of occurrency,positive vocabularies
0,1,aa aaa
1,21,aa batteries
2,1,aa cables
3,1,aa forever
4,1,aa rechargables


Same process occurs for data that have a negative label.

In [4]:
# remove stopwords and create unigram or bigram
vectorizer_neg = CountVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(ngramRange,ngramRange))
y = vectorizer_neg.fit_transform(train_neg.loc[:,"text"])
bow_matrix_neg = y.toarray() # create a bow matrix that keeps occurrency of all words in negative data

neg_vocabularies = vectorizer_neg.get_feature_names() # all words in negative data
numberOfVoc_neg = np.sum(bow_matrix_neg,axis=0) # number of occurrency of vocabularies in negative label
dict_neg = {"negative vocabularies":neg_vocabularies, "number of occurrency":numberOfVoc_neg}
neg_data = pd.DataFrame(dict_neg) # data that have words and their occurrency number
neg_data.head()

,negative vocabularies,number of occurrency
0,aa aaa,2
1,aa batteries,10
2,aa battery,1
3,aa batts,1
4,aa cell,1


For Calculating the accuracy for topics, I created Dataframe with the same method.

Dataframe consist of topics and their number of occurrence.

In [5]:
# keeps topics of positive data and their occurrencies (for prediction according to topic)
vectorizer_pos_topic = CountVectorizer()
z = vectorizer_pos_topic.fit_transform(train_pos.loc[:,"topic"])
bow_matrix_pos_topic = z.toarray()
pos_topics = vectorizer_pos_topic.get_feature_names()
numberOfTopic_pos = np.sum(bow_matrix_pos_topic,axis=0) # number of occurrencies of vocabularies in negative label
dict_pos_topic = {"positive topic":pos_topics, "number of occurrency":numberOfTopic_pos}
pos_topic_data = pd.DataFrame(dict_pos_topic)
pos_topic_data.head()

,number of occurrency,positive topic
0,812,books
1,822,camera
2,802,dvd
3,794,health
4,803,music


In [6]:
# keeps topics of negative data and their occurrencies (for prediction according to topic)
vectorizer_neg_topic = CountVectorizer()
t = vectorizer_neg_topic.fit_transform(train_neg.loc[:,"topic"])
bow_matrix_neg_topic = t.toarray()
neg_topics = vectorizer_neg_topic.get_feature_names()
numberOfTopic_neg = np.sum(bow_matrix_neg_topic,axis=0) # number of occurances of vocabularies in negative label
dict_neg_topic = {"negative topic":neg_topics, "number of occurrency":numberOfTopic_neg}
neg_topic_data = pd.DataFrame(dict_neg_topic)
neg_topic_data.head()

,negative topic,number of occurrency
0,books,811
1,camera,793
2,dvd,787
3,health,772
4,music,775


Naive Bayes Algorithm was implemeted here,

For I dont want to calculate some constant values for each testing, I have calculated them out of the function.
For example, 

'pos_in_all' is ratio of number of samples that have positive label to all data.

'total_pos' is number of samples that have positive label.

'denominator_pos' is used for Laplace Smoothing. If a test word does not exist in the train data, the number of all positive and negative words increases by 1. So, we assume that we have one sample of this test data. Thus, the number of total positive words increases by the length of the positive words list.

After these calculation, Naive Bayes algorithm is ready to run. I used log probability and Laplace Smoothing when I was implementing the Naive Bayes.


In [7]:
import math

# variables that will be need to be able to calculate naive-bayes

# ratio of positive data to all
pos_in_all = sum(numberOfVoc_pos)/(sum(numberOfVoc_pos)+sum(numberOfVoc_neg))
# ratio of negative data to all
neg_in_all = sum(numberOfVoc_neg)/(sum(numberOfVoc_pos)+sum(numberOfVoc_neg))
# total number of positive data
total_pos = sum(numberOfVoc_pos)
# total number of negative data
total_neg = sum(numberOfVoc_neg)
# For Laplace Smoothing, denominators of text and topics
denominator_pos = total_pos+len(pos_vocabularies)
denominator_neg = total_neg+len(neg_vocabularies)
denominator_pos_topics = total_pos+len(pos_topics)
denominator_neg_topics = total_neg+len(neg_topics)

# Naive-Bayes Algorithm Implementation
def naive_bayes(test_vocs,isTopic,isText):

    pos_likelihood = 0
    neg_likelihood = 0

    if isText == 1: # predict according to text
        for voc in test_vocs:
            if (voc not in pos_vocabularies) or (voc not in neg_vocabularies):
                probability_in_pos = math.log(1/denominator_pos)
                probability_in_neg = math.log(1/denominator_neg)
            else:
                # take occurrency of word that the same as test according to its label
                probability_in_pos = math.log(pos_data[pos_data["positive vocabularies"] == voc]["number of occurrency"] / total_pos)
                probability_in_neg = math.log(neg_data[neg_data["negative vocabularies"] == voc]["number of occurrency"] / total_neg)

            neg_likelihood += probability_in_neg
            pos_likelihood += probability_in_pos

    elif isTopic == 1: # predict according to topic
        for voc in test_vocs:
            if (voc not in pos_topics) or (voc not in neg_topics):
                probability_in_pos = math.log(1/denominator_pos_topics)
                probability_in_neg = math.log(1/denominator_neg_topics)
            else:
                probability_in_pos = math.log(pos_topic_data[pos_topic_data["positive topic"] == voc]["number of occurrency"] / total_pos)
                probability_in_neg = math.log(neg_topic_data[neg_topic_data["negative topic"] == voc]["number of occurrency"] / total_neg)

            neg_likelihood += probability_in_neg
            pos_likelihood += probability_in_pos

    final_pos_prob = pos_likelihood+math.log(pos_in_all)
    final_neg_prob = neg_likelihood+math.log(neg_in_all)

    if final_pos_prob>final_neg_prob:
        return "pos"
    else:
        return "neg"


When I run the Naive Bayes, If I want to algorithm predicts sentiment accoring to words, Paremeter 'isText' becomes 1, If I want to algorithm predicts sentiment according to topic, parameter 'isTopic' becomes 1.

After the remove stopwords from text, 1 word remains in some text. This causes an error for bigram model. To be able to handle it, I wrote a try-except part. If lenght of the text is smaller than 2, this document goes to misclassifieds, directly.

In [8]:
correct = 0
false = 0
misclassifieds = [] # list of 5 misclassified datas
for index in test.index:
    vectorizer_test = CountVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(ngramRange,ngramRange))
    try:
        u = vectorizer_test.fit_transform([test.loc[index,"text"]])
    except ValueError:
        misclassifieds.append(index)
        continue
    words = vectorizer_test.get_feature_names()
    bow = u.toarray()
    for i in range(len(bow[0])):
        if bow[0][i] > 1:
            for j in range(bow[0][i]-1):
                words.append(words[i])

    prediction = naive_bayes(words,0,1) # predict according to text
    if prediction == test.loc[index,"label"]:
        correct += 1
    else:
        false += 1
        if len(misclassifieds) < 6:
            misclassifieds.append(index)

print("Some Misclassifieds:")
for i in misclassifieds:
    print(test.loc[i,"doc_identifier"],":",test.loc[i,"label"],test.loc[i,"text"])

accuracy = 100*correct/(correct+false)

Some Misclassifieds:
444.txt : pos the series just keeps on getting better the bob newhart show was one of the best comedies ever on television the other funny show was newhart which i cannot understand why it has not been released on dvd the whole cast just clicks i wish shows like this were still on television if you enjoy a good laugh then buy this and the other sets
460.txt : pos this is one of the better historical dramas from the many are stagey and slow and while this production of madame bovary is very much constrained to stay indoors this works to advantage for this story of a woman who feels so trapped by her life and her world emma bovary is not a very sympathetic creature she is married to a man who loves her with all his heart who tries to give her everything she wants is willing to ruin himself to make her happy and she still cheats on him and remains miserable in short she is selfish and inconsiderate as ugly inside as she is pretty on the outside francesca annis plays e

- Error Analysis:

For example in 351.txt, 'Not' and 'uncommon' are negatively effective words but when they use together, it has positive meaning. This stuation can create complication

Accuracy using Bigram Model:

In [9]:
print("Accuracy of prediction using Bigram: ",accuracy)

Accuracy of prediction using Bigram:  65.23929471032746


Accuracy using Unigram Model: %80

My implementation works better in Unigram model.

In [10]:
# Prediction From Topic

correct = 0
false = 0

for index in test.index:
    words = [test.loc[index,"topic"]]

    prediction = naive_bayes(words,1,0)
    if prediction == test.loc[index,"label"]:
        correct += 1
    else:
        false += 1

accuracy = 100*correct/(correct+false)
print("Accuracy of prediction from topic: ",accuracy)


Accuracy of prediction from topic:  48.38438942509442


I see that prediction the sentiment using topic of document is not a good way.

- Analyzing effect of the stopwords

Removing stop words speed up as well as improve the accuracy because stopwords are existing both of the documents that has label negative and positive. So it is usefull but in bigram model, when I remove the stopwords from document, many prepositions are deleted. So, juxtaposed words became meaningless. This can be reason of bigram model has less accuracy from unigram has.